In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from datetime import date

In [2]:
# Function to focus on a political party and create a dataframe with the agreement score of each deputy
def voteType(df):
    
    # Count the number of times each voting orientation occurred
    voting = df["new"].value_counts()
  
    ## Calculate the percentage of deputies who vote according to the party sum of all times 
    ## they agree divided by the number of times they voted => percentage
    percent = (df.groupby('deputado_nome')['new'].sum())/df['deputado_nome'].value_counts()
    pdAcord = pd.DataFrame({'deputado_nome':df['deputado_nome'].unique().tolist(),'score_accordingly': round(percent*100,2)})
    
    # Merge the calculated agreement score with the original dataframe
    dfPL = pd.merge(df,pdAcord, on ='deputado_nome')
    
    # Create encoder and label encoder objects
    encoder = OneHotEncoder() 
    label_encoder = LabelEncoder()
    
    # Encode the deputy names, birthplace, and voting orientation
    encoded_deputados = label_encoder.fit_transform(dfPL['deputado_nome'])
    encoded_gender = label_encoder.fit_transform(dfPL['siglaSexo'])
    encoded_municipios = label_encoder.fit_transform(dfPL['municipioNascimento'])
    encoded_orientacao = label_encoder.fit_transform(dfPL['orientacao'])
    encoded_vote = label_encoder.fit_transform(dfPL['voto'])

    
    
    # Add the encoded values to the dataframe
    dfPL['deputado_enc'] = encoded_deputados
    dfPL['orientacao_enc'] = encoded_orientacao
    dfPL['gender_enc'] = encoded_gender
    dfPL['vote_enc'] = encoded_vote
    
    return dfPL

In [3]:
df = pd.read_csv('dataVoting.csv')
dfP = voteType(df)

In [4]:
dfP[['deputado_enc', 'idVotacao', 'vote_enc']].value_counts().to_frame().rename(columns={0:'repetitions'})

repetitions
deputado_enc idVotacao  vote_enc             
316          2301321-40 4                  28
323          2301321-40 4                  28
177          2301321-40 4                  28
173          2301321-40 4                  28
248          2301321-40 4                  28
...                                       ...
116          2334096-29 4                   3
42           2334096-29 4                   3
             2334095-31 4                   2
161          2334095-31 4                   2
143          2334095-31 4                   2

[112156 rows x 1 columns]

### O resultado da célula anterior demonstra que literalmente todas as linhas do dataframe dfP (considerando apenas as colunas deputado_enc, idVotacao, e vote_enc) não são únicas, mas se repetem desde duas até 28 vezes. Isto pode provocar um data leakage dado que ao fazer o train-test split, eventualmente algumas das linhas no conjunto de teste também estariam presentes no conjunto de treinamento, tornando sua previsão trivial.

In [5]:
df = pd.read_csv("votacoes-2022.csv", sep=';')
dfV = pd.read_csv("votacoesVotos-2022.csv", sep=';')
dfO =  pd.read_csv("votacoesOrientacoes-2022.csv", sep=';')
deputados = pd.read_csv("deputados.csv", sep=';')

themes =  pd.read_csv("proposicoesTemas-2022.csv", sep=';')

In [6]:
df = df.drop(['uri', 'data', 'dataHoraRegistro', 'idOrgao', 'uriOrgao',
         'uriEvento','descricao' , 'ultimaAberturaVotacao_dataHoraRegistro',
       'ultimaAberturaVotacao_descricao',
       'ultimaApresentacaoProposicao_dataHoraRegistro',
       'ultimaApresentacaoProposicao_descricao',
       'ultimaApresentacaoProposicao_idProposicao',
       'ultimaApresentacaoProposicao_uriProposicao'], axis=1)

In [7]:
df.columns = ['idVotacao', 'siglaOrgao', 'idEvento', 'aprovacao', 'votosSim', 'votosNao',
       'votosOutros']

##Retirar variaveis que nao vao ser usadas para estudar;
df = df.drop([ 'votosSim', 'votosNao','votosOutros'], axis=1)
df

,idVotacao,siglaOrgao,idEvento,aprovacao
0,2276475-116,PLEN,64635,1.0
1,2236607-112,PLEN,64635,0.0
2,2276475-96,PLEN,64635,0.0
3,2276475-107,PLEN,64635,1.0
4,2276475-111,PLEN,64635,1.0
...,...,...,...,...
3834,2314962-21,SECAP(SGM),0,1.0
3835,2314962-22,SECAP(SGM),0,1.0
3836,2314962-23,SECAP(SGM),0,1.0
3837,2314962-24,PLEN,0,1.0


In [8]:
# deputados.columns
deputados = deputados.drop(['uri', 'nomeCivil', 'cpf', 'urlRedeSocial', 'urlWebsite', 'ufNascimento'], axis=1)

In [9]:
today = date.today()
today.year

##PARA DEPUTADOS APARTIR DA LEGISLATURA 54º
deputados = deputados[deputados["idLegislaturaInicial"] > 54]
deputados.loc[deputados["dataFalecimento"].isna(), "dataFalecimento"] = today

deputados["dataFalecimento"] = pd.to_datetime(deputados["dataFalecimento"], errors = 'coerce')
deputados["dataNascimento"] = pd.to_datetime(deputados["dataNascimento"], errors = 'coerce')
deputados["idade"] =  ((deputados["dataFalecimento"] - deputados["dataNascimento"]).astype('<m8[Y]') )
deputados = deputados.drop(['dataFalecimento', 'dataNascimento'], axis=1)

In [10]:
deputados.columns = ['deputado_nome', 'idLegislaturaInicial', 'idLegislaturaFinal', 'siglaSexo',
       'municipioNascimento', 'idade']
deputados

,deputado_nome,idLegislaturaInicial,idLegislaturaFinal,siglaSexo,municipioNascimento,idade
0,Abílio Santana,56,56,M,Salvador,58.0
1,Abou Anni,56,56,M,São Paulo,56.0
2,Acácio Favacho,56,56,M,Macapá,39.0
3,Adolfo Viana,56,56,M,Salvador,42.0
4,Adriana Ventura,56,56,F,São Paulo,54.0
...,...,...,...,...,...,...
931,WILSON BESERRA,55,55,M,Rio de Janeiro,53.0
934,XUXU DAL MOLIN,55,55,M,Realeza,47.0
937,ZECA CAVALCANTI,55,55,M,Arcoverde,56.0
938,ZECA DO PT,55,55,M,Porto Murtinho,73.0


In [11]:
df_resumo = pd.merge(df,dfV, on ='idVotacao')
df_resumo = pd.merge(df_resumo, deputados, on ='deputado_nome')
df_resumo = pd.merge(df_resumo,dfO, on ='idVotacao')
df_resumo[['deputado_id', 'idVotacao', 'voto']].value_counts().to_frame().rename(columns={0:'repetitions'})

repetitions
deputado_id idVotacao   voto             
204537      2297579-87  Sim            28
193726      2265631-107 Não            28
            2265631-116 Sim            28
            2265631-109 Não            28
178857      2285889-205 Sim            28
...                                   ...
204572      2334096-29  Sim             3
66828       2334096-29  Sim             3
204572      2334095-31  Sim             2
204419      2334095-31  Sim             2
204533      2334095-31  Sim             2

[112156 rows x 1 columns]

In [12]:
df_resumo = pd.merge(df,dfV, on ='idVotacao')
df_resumo = pd.merge(df_resumo, deputados, on ='deputado_nome')
#df_resumo = pd.merge(df_resumo,dfO, on ='idVotacao')
df_resumo[['deputado_id', 'idVotacao', 'voto']].value_counts().to_frame().rename(columns={0:'repetitions'})

repetitions
deputado_id idVotacao  voto             
66179       105464-319 Não             1
204473      2299480-8  Sim             1
            2306112-69 Sim             1
            2306112-62 Sim             1
            2306112-59 Sim             1
...                                  ...
194260      2318736-85 Não             1
            2318736-82 Sim             1
            2318736-79 Não             1
            2318736-62 Não             1
220008      596844-116 Sim             1

[112180 rows x 1 columns]

#### O pd.merge de df_resumo com dfO considerando idVotação seria o causador das repetições. Isto ocorre pois em cada votação podem haver diversas orientações, feitas por diferentes partidos, blocos partidários, bancadas etc. Por isso, quando este merge é realizado, cada voto que constava em df_resumo acaba por ser combinado com cada uma das orientações dadas para a votação a que o voto corresponde, gerando assim as cópias de votos que foram percebidas. Aparentemente não há qualquer prejuízo em não realizar o merge de dfO, sendo esta talvez a solução mais simples esta questão.

In [13]:
dfO

,idVotacao,uriVotacao,siglaOrgao,descricao,siglaBancada,uriBancada,orientacao
0,2314962-33,https://dadosabertos.camara.leg.br/api/v2/vota...,PLEN,Aprovado o Substitutivo ao Projeto de Lei Comp...,Maioria,NaN,Sim
1,2314962-33,https://dadosabertos.camara.leg.br/api/v2/vota...,PLEN,Aprovado o Substitutivo ao Projeto de Lei Comp...,Novo,https://dadosabertos.camara.leg.br/api/v2/part...,Sim
2,2314962-33,https://dadosabertos.camara.leg.br/api/v2/vota...,PLEN,Aprovado o Substitutivo ao Projeto de Lei Comp...,PP,https://dadosabertos.camara.leg.br/api/v2/part...,Sim
3,2314962-33,https://dadosabertos.camara.leg.br/api/v2/vota...,PLEN,Aprovado o Substitutivo ao Projeto de Lei Comp...,PDT,https://dadosabertos.camara.leg.br/api/v2/part...,Sim
4,2314962-33,https://dadosabertos.camara.leg.br/api/v2/vota...,PLEN,Aprovado o Substitutivo ao Projeto de Lei Comp...,Solidaried,NaN,Sim
...,...,...,...,...,...,...,...
12034,2312874-9,https://dadosabertos.camara.leg.br/api/v2/vota...,PLEN,Aprovado o Requerimento de Urgência (Art. 155 ...,Solidaried,NaN,Sim
12035,2312874-9,https://dadosabertos.camara.leg.br/api/v2/vota...,PLEN,Aprovado o Requerimento de Urgência (Art. 155 ...,Novo,https://dadosabertos.camara.leg.br/api/v2/part...,Sim
12036,2312874-9,https://dadosabertos.camara.leg.br/api/v2/vota...,PLEN,Aprovado o Requerimento de Urgência (Art. 155 ...,PT,https://dadosabertos.camara.leg.br/api/v2/part...,Obstrução
12037,2312874-9,https://dadosabertos.camara.leg.br/api/v2/vota...,PLEN,Aprovado o Requerimento de Urgência (Art. 155 ...,PP,https://dadosabertos.camara.leg.br/api/v2/part...,Sim


In [ ]:
##find values in cols where orientation and vote is equal
## if the value matches => 1 and if its different => 0
df2["new"] = np.where((df2['voto'] == df2['orientacao']), 1, 0)